# Defining Path


In [ ]:
import sys

sys.path.append('..')
sys.path.append('../examples')

# Importing Libraries

In [ ]:
# Pytorch 
import torch
import torchvision
# General Libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import re
import random

# System and Path Libraries
import os
from pathlib import Path

# Libraries for Tranforming into Yolov7 format data
from train_cars import CarsDatasetAdaptor
from yolov7.dataset import Yolov7Dataset
from yolov7.dataset import create_yolov7_transforms
from yolov7.plotting import show_image
# Train Test Split
from sklearn.model_selection import train_test_split

# Libraries for Training:
from functools import partial
from func_to_script import script
from PIL import Image
from pytorch_accelerated.callbacks import (
    ModelEmaCallback,
    ProgressBarCallback,
    SaveBestModelCallback,
    get_default_callbacks,
    EarlyStoppingCallback
)
from pytorch_accelerated.schedulers import CosineLrScheduler
from torch.utils.data import Dataset
from yolov7 import create_yolov7_model
from yolov7.dataset import (
    Yolov7Dataset,
    create_base_transforms,
    create_yolov7_transforms,
    yolov7_collate_fn,
)
from yolov7.evaluation import CalculateMeanAveragePrecisionCallback
from yolov7.loss_factory import create_yolov7_loss
from yolov7.mosaic import MosaicMixupDataset, create_post_mosaic_transform
from yolov7.trainer import Yolov7Trainer, filter_eval_predictions
from yolov7.utils import SaveBatchesCallback, Yolov7ModelEma

# FLOPs calculation Libraries
from thop import profile
from thop import clever_format

######################
%load_ext autoreload
%autoreload 2

# Printing the versions
print(torch.__version__)
print(torchvision.__version__)

# Data Loading

In [ ]:
# Change the file names as needed
main_folder= "anatomical-model-papilla-yolo-export-v1_dheeraj"
images_folder= 'Aslanian_Exalt_1_images'
annotations_file_name= "output_coco_annotations.json"

In [ ]:
##############################################################
data_path = "../data/papilla/" + main_folder
data_path = Path(data_path)
images_path = data_path / images_folder # all_images
annotations_file_path = data_path / annotations_file_name # annotations_final.csv

# Crop the Images 

In [ ]:
import cv2

images_list= os.listdir(images_path)## Creatng list of images from the actual image directory
print(f"Number of Images:{len(images_list)}")
# Verify coordinate values--> Change these values according to the image
####(tip- can use paint app to find the xmin,ymin,xmax,ymax)

xmin = 560  # example xmin value
ymin = 60  # example ymin value
xmax = 1680  # example xmax value
ymax = 1027  # example ymax value

images_list = os.listdir(images_path)
## Defining the directory for croped images
cropped_images_dir  = data_path / str("cropped_images-of-"+str(images_path).split("\\")[-1]) 

if not os.path.exists(cropped_images_dir):
    os.makedirs(cropped_images_dir)
    print(f"Created directory: {cropped_images_dir}")
    
    for image_name in images_list:
        # Load the image
        image_path = images_path / image_name
        
        image = cv2.imread(str(image_path))

        # Perform cropping
        cropped_image = image[ymin:ymax, xmin:xmax]
        # Save the cropped image
        cropped_image_path = cropped_images_dir / image_name
        cv2.imwrite(str(cropped_image_path), cropped_image)
#         print(f"Cropped image saved: {cropped_image_path}")
#         break  # Uncomment this line if you want to crop only the first image for testing purposes

else:
    print(f"Directory already exists: {cropped_images_dir}")


## Reading the Json file, Updating Bounding Box, Creating Dataframe

### This function is used to get dataframe while using Coco-Annotator

In [ ]:
# I have the filename: Extracting other information from data['annotations']
#But file name is present in data['images']
# We need to search for the ID in data['images'] wrt to the filename
# Then map each id with its respective image_id in data['annotations']--> from here we can extarct the information 
#LIke bbox, category_id 
# Updating the Bounding boxes and Creating the Dataframe
def get_dataframe_coco_annotator(annotations_file_path):
    # Reading the Annotation File
    with open(annotations_file_path, 'r') as f:
            data = json.load(f)
            
    papilla_images_list = os.listdir(images_path)

    ## Creating dataframe    
    my_df= {'image':[], 'xmin':[],'ymin':[], 'xmax':[], 'ymax':[], 'class_name':[], 'has_annotation':[], 'image_id':[], 'class_id':[]}
    for filename in papilla_images_list:
        my_id= [d for d in data['images'] if d['file_name']== filename][0]['id']
        my_data= [d for d in data['annotations'] if d['image_id']== my_id] 
        filtered_data = [d for d in my_data if d['category_id'] != 216][0]    
    #     print(filtered_data)
        my_df['image'].append(filename)
        my_df['image_id'].append(np.nan)

        if filtered_data['category_id']==168:

            my_df['class_name'].append('background')
            my_df['class_id'].append(np.nan)
            my_df['has_annotation'].append(False)## Im assuming if its a background (category id 168) there is no annotation

            my_df['xmin'].append(np.nan)
            my_df['ymin'].append(np.nan)
            my_df['xmax'].append(np.nan)
            my_df['ymax'].append(np.nan)

        else:
            # Reading Bounding Box
            original_bbox= filtered_data['bbox']# The bbox is in the format [xmin, ymin, w, h]
    ##################################################################################################        
            # Converting bbox according to the cropped image and converting the format to [xmin, ymin, xmax, ymax]
            crop_width = xmax - xmin
            crop_height = ymax - ymin

            new_xmin = max(0, original_bbox[0] - xmin)
            new_ymin = max(0, original_bbox[1] - ymin)
            new_xmax = min(crop_width, original_bbox[0] + original_bbox[2] - xmin)
            new_ymax = min(crop_height, original_bbox[1] + original_bbox[3] - ymin)
    ################################################################################################

            my_df['class_name'].append('papilla')
            my_df['class_id'].append(0.0)
            my_df['has_annotation'].append(True)

            my_df['xmin'].append(new_xmin)
            my_df['ymin'].append(new_ymin)
            my_df['xmax'].append(new_xmax)# The bounding boxes are in the format x,y,w,g, from coco annotater
            my_df['ymax'].append(new_ymax)# Hence converting w & h to xmax, ymax 
    
    final_df=  pd.DataFrame.from_dict(my_df)
    final_df.image_id= list(range(1, len(final_df)+1))
    return final_df

### This function is used to get dataframe while using V7 Annotator- coco annotations

In [ ]:
# Updating the Bounding boxes and Creating the Dataframe
def get_dataframe_v7_annotator(annotations_file_path):
    with open(annotations_file_path, 'r') as f:
            data = json.load(f)
    annotations= data['annotations']        
    my_images_list = os.listdir(data_path / "Aslanian_Exalt_1_images")

    #Creating dataframe    

    my_df= {'image':[], 'xmin':[],'ymin':[], 'xmax':[], 'ymax':[], 'class_name':[], 
            'has_annotation':[], 'image_id':[], 'class_id':[] }
    # Iterate over the images
    for image_filename in my_images_list:
        # Find the annotation for the current image filename
        new_image_filename = int(image_filename.split('.')[0])
        matching_data= [d for d in data['annotations'] if d['image_id']== new_image_filename]
        
        ## Inputting data into dictionary for the dataframe building
        my_df['image'].append(image_filename)
        my_df['image_id'].append(np.nan)

        if len(matching_data)==0:
            my_df['class_name'].append('background')
            my_df['class_id'].append(np.nan)
            my_df['has_annotation'].append(False)## Im assuming if its a background (category id 168) there is no annotation

            my_df['xmin'].append(np.nan)
            my_df['ymin'].append(np.nan)
            my_df['xmax'].append(np.nan)
            my_df['ymax'].append(np.nan)

        else:
            # Reading Bounding Box
            original_bbox= matching_data[0]['bbox']# The bbox is in the format [xmin, ymin, w, h]
    ##################################################################################################        
            # Converting bbox according to the cropped image and converting the format to [xmin, ymin, xmax, ymax]
            crop_width = xmax - xmin
            crop_height = ymax - ymin

            new_xmin = max(0, original_bbox[0] - xmin)
            new_ymin = max(0, original_bbox[1] - ymin)
            new_xmax = min(crop_width, original_bbox[0] + original_bbox[2] - xmin)
            new_ymax = min(crop_height, original_bbox[1] + original_bbox[3] - ymin)
    ################################################################################################

            my_df['class_name'].append('papilla')
            my_df['class_id'].append(0.0)
            my_df['has_annotation'].append(True)

            my_df['xmin'].append(new_xmin)
            my_df['ymin'].append(new_ymin)
            my_df['xmax'].append(new_xmax)# The bounding boxes are in the format x,y,w,g, from coco annotater
            my_df['ymax'].append(new_ymax)# Hence converting w & h to xmax, ymax 
    
    final_df=  pd.DataFrame.from_dict(my_df)
    final_df.image_id= list(range(1, len(final_df)+1))
    return final_df

### Call anyone of the above two functions based on the type of annotation file being used: COCO or V7 COCO

In [ ]:
annotations_df= get_dataframe_v7_annotator(annotations_file_path)

In [ ]:
# annotations_df= get_dataframe_coco_annotator(annotations_file_path)

In [ ]:
annotations_df.nunique()

In [ ]:
annotations_df

## Now continuing using the cropped image path and the annotation (dataframe)

In [ ]:
def load_cars_df(df):
        
    image_id_to_image = {i: im for i, im in zip(df.image_id, df.image)}
    image_to_image_id = {v: k for k, v, in image_id_to_image.items()}
    
    class_id_to_label = dict(
        enumerate(df.query("has_annotation == True").class_name.unique())
    )
    class_label_to_id = {v: k for k, v in class_id_to_label.items()}
        
    # Splitting Data into 70-15-15--> Train, Validation and Test
    train_df, valid_test_df = train_test_split(df, test_size=0.3, random_state=42)
    valid_df, test_df = train_test_split(valid_test_df, test_size=0.5, random_state=42)

    lookups = {
        "image_id_to_image": image_id_to_image,
        "image_to_image_id": image_to_image_id,
        "class_id_to_label": class_id_to_label,
        "class_label_to_id": class_label_to_id,
    }
    return train_df, valid_df, test_df, lookups

We can now use this function to load our data:

In [ ]:
train_df, valid_df, test_df, lookups = load_cars_df(annotations_df)

In [ ]:
train_df.head()

In [ ]:
valid_df.head()

In [ ]:
test_df.head()

In [ ]:
print(train_df.image.nunique(), valid_df.image.nunique(), test_df.image.nunique())

In [ ]:
lookups.keys()

In [ ]:
lookups['class_label_to_id'], lookups['class_id_to_label']

In [ ]:
print(f"Num. annotated images in training set: {len(train_df.query('has_annotation == True').image.unique())}")
print(f"Num. Background images in training set: {len(train_df.query('has_annotation == False').image.unique())}")
print(f"Total Num. images in training set: {len(train_df.image.unique())}")
print('------------')

print(f"Num. annotated images in validation set: {len(valid_df.query('has_annotation == True').image.unique())}")
print(f"Num. Background images in validation set: {len(valid_df.query('has_annotation == False').image.unique())}")
print(f"Total Num. images in validation set: {len(valid_df.image.unique())}")

# Create a Dataset Adaptor

In [ ]:
train_ds = CarsDatasetAdaptor(cropped_images_dir, train_df)
valid_ds= CarsDatasetAdaptor(cropped_images_dir, valid_df)
test_ds= CarsDatasetAdaptor(cropped_images_dir, test_df)

In [ ]:
train_ds

# Transforms

In [ ]:
target_image_size = 640

In [ ]:
train_yds = Yolov7Dataset(train_ds, transforms=create_yolov7_transforms(image_size=(target_image_size, target_image_size)))
eval_yds= Yolov7Dataset(valid_ds, transforms=create_yolov7_transforms(image_size=(target_image_size, target_image_size)))
test_yds= Yolov7Dataset(test_ds, transforms=create_yolov7_transforms(image_size=(target_image_size, target_image_size)))

# Visualization

In [ ]:
idx = 16
    
image_tensor, labels, image_id, image_size = train_yds[idx]

print(f'Image: {image_tensor.shape}')
print(f'Labels: {labels}')

# denormalize boxes
boxes = labels[:, 2:]
boxes[:, [0, 2]] *= target_image_size #image_size[1]## Multiplying with targetimagesize becasue, padding was applied (using transforms above)
boxes[:, [1, 3]] *= target_image_size #image_size[0]

show_image(image_tensor.permute( 1, 2, 0), boxes.tolist(), [lookups['class_id_to_label'][int(c)] for c in labels[:, 1]], 'cxcywh')
plt.show()
print(f'Image id: {image_id}')
print(f'Image size: {image_size}')

# Training

In [ ]:
# CHangint the path: this is the path wehre the trained weight (.pt) file is created
%cd "C:\Users\endo\Desktop\Yolov7-training-main\Yolov7-training-main\examples\"

# Training code

## Note: Note that each time the trainign is run: a file named: "best_model.pt" is created so make sure to rename is before running the training- otherwise it gets overwritten. Both for finetuning and training from scratch

## Training from Scratch Function

In [ ]:
# Training from Scratch
DATA_PATH =data_path

def train_scratch(
    train_ds, valid_ds,
    data_path: str = DATA_PATH,
    image_size: int = 640,
    pretrained: bool = False,
    num_epochs: int = 50, #was 50
    batch_size: int = 8,
    device: str = 'cuda' if torch.cuda.is_available() else 'cpu',# Add this line to set the device to use
):
##############################################################################
    # CHecking on which device the training is going to run
    print(device)
    #Code added by Mike: print name of GPU
    print(torch.cuda.get_device_name(torch.cuda.current_device()))
##############################################################################
# Defining Number of classes
    num_classes= 1 
##############################################################################
    # Mosaic Augmentation
    mds = MosaicMixupDataset(
        train_ds,
        apply_mixup_probability=0.15,
        post_mosaic_transforms=create_post_mosaic_transform(
            output_height=image_size, output_width=image_size
        ),
    )
##############################################################################
    if pretrained:
        # disable mosaic if finetuning
        mds.disable()
        print('Mosaic Disabled')
##############################################################################
    # Creating the Yolo Transformed data 
    train_yds = Yolov7Dataset(
        mds,
        create_yolov7_transforms(training=True, image_size=(image_size, image_size)),
    )
    eval_yds = Yolov7Dataset( 
        valid_ds,
        create_yolov7_transforms(training=False, image_size=(image_size, image_size)),
    )
##############################################################################
    # create model, loss function and optimizer
    model = create_yolov7_model(
        architecture="yolov7", num_classes=num_classes, pretrained=pretrained
    ).to(device) # Moving the model to device
##############################################################################
    param_groups = model.get_parameter_groups()
##############################################################################
    loss_func = create_yolov7_loss(model, image_size=image_size)
##############################################################################
    optimizer = torch.optim.SGD(
        param_groups["other_params"], lr=0.01, momentum=0.937, nesterov=True
    )
##############################################################################
    # create evaluation callback and trainer
    calculate_map_callback = (
        CalculateMeanAveragePrecisionCallback.create_from_targets_df(
            targets_df=valid_df.query("has_annotation == True")[
                ["image_id", "xmin", "ymin", "xmax", "ymax", "class_id"]
            ],
            image_ids=set(valid_df.image_id.unique()),
            iou_threshold=0.2,
        )
    )
##############################################################################
    trainer = Yolov7Trainer(
        model=model,
        optimizer=optimizer,
        loss_func=loss_func,
        filter_eval_predictions_fn=partial(
            filter_eval_predictions, confidence_threshold=0.01, nms_threshold=0.3
        ),
        callbacks=[
            calculate_map_callback,
            SaveBestModelCallback(watch_metric="map", greater_is_better=True),
            SaveBatchesCallback("./batches", num_images_per_batch=3),
            *get_default_callbacks(progress_bar=True),
        ],
    )
##############################################################################
    # calculate scaled weight decay and gradient accumulation steps
    total_batch_size = (
        batch_size * trainer._accelerator.num_processes
    )  # batch size across all processes

    nominal_batch_size = 64
    num_accumulate_steps = max(round(nominal_batch_size / total_batch_size), 1)
    base_weight_decay = 0.0005
    scaled_weight_decay = (
        base_weight_decay * total_batch_size * num_accumulate_steps / nominal_batch_size
    )

    optimizer.add_param_group(
        {"params": param_groups["conv_weights"], "weight_decay": scaled_weight_decay}
    )
##############################################################################
    # run training
    trainer.train(
        num_epochs=num_epochs,
        train_dataset=train_yds,
        eval_dataset=eval_yds,
        per_device_batch_size=batch_size,
        create_scheduler_fn=CosineLrScheduler.create_scheduler_fn(
            num_warmup_epochs=5,
            num_cooldown_epochs=5,
            k_decay=2,
        ),
        collate_fn=yolov7_collate_fn,
        gradient_accumulation_steps=num_accumulate_steps,
    )
##############################################################################
## NOTE##
# The mAP obtained here for the Validation set is when IoU is set for 0.2
# Difference between finetuning and training from scratch code:
# 1. Mosaic Augmentation is not applied in Finetuning
# 2. Optimizer: model.get_parameter_groups() in Training from scratch; model.parameters() in FInetuning

## Finetuning Function

In [ ]:
# Finetuning the model
DATA_PATH =data_path

def train_finetune(
    train_ds, valid_ds,
    data_path: str = DATA_PATH,
    image_size: int = 640,
    pretrained: bool = True,
    num_epochs: int = 50,
    batch_size: int = 8,
    device: str = 'cuda' if torch.cuda.is_available() else 'cpu',# Add this line to set the device to use
):
##############################################################################
    # CHecking on which device the training is going to run
    print(device)
    #Code added by Mike: print name of GPU
    print(torch.cuda.get_device_name(torch.cuda.current_device()))
##############################################################################
# Defining Number of classes
    num_classes= 1
##############################################################################
    train_yds = Yolov7Dataset(
        train_ds,
        create_yolov7_transforms(training=True, image_size=(image_size, image_size)),
    )
    eval_yds = Yolov7Dataset(
        valid_ds,
        create_yolov7_transforms(training=False, image_size=(image_size, image_size)),
    )
##############################################################################
    # Create model, loss function and optimizer
    model = create_yolov7_model(
        architecture="yolov7", num_classes=num_classes, pretrained=pretrained
    ).to(device) # Dheeraj added .to(torch.device("cpu")) 
##############################################################################

    loss_func = create_yolov7_loss(model, image_size=image_size)
    
##############################################################################
    optimizer = torch.optim.SGD(
        model.parameters(), lr=0.01, momentum=0.9, nesterov=True
    )
##############################################################################
    # create evaluation callback and trainer
    calculate_map_callback = (
        CalculateMeanAveragePrecisionCallback.create_from_targets_df(
            targets_df=valid_df.query("has_annotation == True")[
                ["image_id", "xmin", "ymin", "xmax", "ymax", "class_id"]
            ],
            image_ids=set(valid_df.image_id.unique()),
            iou_threshold=0.2,
        )
    )
##############################################################################
    # Create trainer and train
    trainer = Yolov7Trainer(
        model=model,
        optimizer=optimizer,
        loss_func=loss_func,
        filter_eval_predictions_fn=partial(
            filter_eval_predictions, confidence_threshold=0.01, nms_threshold=0.3
        ),
        callbacks=[
            calculate_map_callback,
            SaveBestModelCallback(watch_metric="map", greater_is_better=True),
            EarlyStoppingCallback(
                early_stopping_patience=3,
                watch_metric="map",
                greater_is_better=True,
                early_stopping_threshold=0.001,
            ),
            *get_default_callbacks(progress_bar=True),
        ],
    )
#############################################################################
    trainer.train(
        num_epochs=num_epochs,
        train_dataset=train_yds,
        eval_dataset=eval_yds,
        per_device_batch_size=batch_size,
        create_scheduler_fn=CosineLrScheduler.create_scheduler_fn(
            num_warmup_epochs=5,
            num_cooldown_epochs=5,
            k_decay=2,
        ),
        collate_fn=yolov7_collate_fn,
    )

## Based on the type of training call that function: train_scratch or train_finetune

In [ ]:
# train_finetune(train_ds, valid_ds)

In [ ]:
train_scratch(train_ds, valid_ds)

# Difference between Finetune and Training from Scratch

The notable differences between the "Training from Scratch" and "Fine-tuning" scenarios in the provided script:

Fine-tuning the model:

Dataset: The script defines train_yds and eval_yds using the Yolov7Dataset and create_yolov7_transforms functions for both training and evaluation datasets.

Optimizer: The optimizer is created using model.parameters() instead of model.get_parameter_groups().
Early Stopping: The script includes the EarlyStoppingCallback in the list of trainer callbacks, which helps stop training early if the metric does not improve.

No Gradient Accumulation Steps: Unlike the "Training from Scratch" script, there is no calculation or mention of gradient accumulation steps in the "Fine-tuning" script.

# Calculating mAP, Precision and Recall on Test set

## ***Note***
#### I have created my own code for calculation of mAP, Precison, Recall, and Precision-Recall Curve
#### But this might not be necessary if we can use the functions provided in the evaluation folder correctly
#### Path for evaluation folder: C:\Users\endo\Desktop\Yolov7-training-main\Yolov7-training-main\yolov7\evaluation
#### Some idea on how to use it can be learnt from the above cell (Training Code), where the callbacks for calcualtion mAP were used

In [ ]:
# Defining model 
best_model = create_yolov7_model('yolov7', num_classes=1)
best_model.eval();

In [ ]:
# Change the weights file name as needed
model_weights_name= "anatomical_data_Finetune_best_model.pt"

In [ ]:
# Loading Weights
best_model_path= 'C:\\Users\\endo\\Desktop\\Yolov7-training-main\\Yolov7-training-main\\examples\\'+ model_weights_name
checkpoint = torch.load(best_model_path)
state_dict = checkpoint['model_state_dict']
best_model.load_state_dict(state_dict)

# FLOPS Calculation

In [ ]:
# Create a sample input tensor
input_tensor = torch.randn(1, 3, 640, 640)  # Assuming input shape of (batch_size, channels, height, width)
# Calculating
macs, params = profile(best_model, inputs=(input_tensor,))
# Because 1 MACS equals roughly 2 FLOPs
flops= macs/2
# Changing format
flops, params = clever_format([flops, params], "%.3f")
print(f"FLOPs: {flops}")

## Running inference on test_yds

In [ ]:
for idx in range(len(test_yds)):
    image_tensor, labels, image_id, image_size = test_yds[idx]
    with torch.no_grad():
        model_outputs = best_model(image_tensor[None])
        preds = best_model.postprocess(model_outputs, conf_thres=0., multiple_labels_per_box=False)

        # Inference
        nms_predictions = filter_eval_predictions(preds, confidence_threshold=0.1)
        nms_predictions[0].shape
        pred_boxes = nms_predictions[0][:, :4]
        class_ids = nms_predictions[0][:, -1]

        show_image(image_tensor.permute( 1, 2, 0), pred_boxes.tolist(), class_ids.tolist())
        plt.show()
#         print(f'Image id: {image_id}')
#         print(f'Original Image size: {image_size}')
#         print(f'Resized Image size: {image_tensor.shape[1:]}')
    break

In [ ]:
def intersection_over_union(boxes_preds, boxes_labels, box_format= "corners"):
    # (N,4): N--number of bboxes
    # boxes_labels shape is (N,4)
    
    '''Calculates intersection over union
    Parameters:
        boxes_preds (tensor): Predictions of Bounding Boxes (BATCH_SIZE, 4)
        boxes_labels (tensor): Correct Labels of Boxes (BATCH_SIZE, 4)
        box_format (str): midpoint/corners, if boxes (x,y,w,h) or (x1,y1,x2,y2)
    Returns:
        tensor: Intersection over union for all examples
        '''       
    if box_format == "corners":
        
        #Converting cx,cy,w,h (center_x, center_y,w,h) into (xmin, ymin, xmax, ymax)
        
#         print(boxes_labels)
#         print(boxes_preds)
        
        center_x= boxes_labels[..., 0:1]
        center_y = boxes_labels[..., 1:2] 
        width = boxes_labels[..., 2:3] 
        height= boxes_labels[..., 3:4] 
        new_boxes_labels = torch.zeros_like(boxes_labels) # Initializing the tensor

        new_boxes_labels[..., 0:1]= center_x - (width / 2)
        new_boxes_labels[..., 1:2]= center_y - (height / 2)
        new_boxes_labels[..., 2:3]= center_x + (width / 2)
        new_boxes_labels[..., 3:4]= center_y + (height / 2)        
        
        
        pred_xmin= boxes_preds[..., 0:1]
        pred_ymin= boxes_preds[..., 1:2]
        pred_xmax= boxes_preds[..., 2:3]
        pred_ymax= boxes_preds[..., 3:4]

        label_xmin= new_boxes_labels[..., 0:1]
        label_ymin= new_boxes_labels[..., 1:2]
        label_xmax= new_boxes_labels[..., 2:3]
        label_ymax= new_boxes_labels[..., 3:4]

    inter_area = max(0, min(pred_xmax, label_xmax) - max(pred_xmin, label_xmin)) * \
             max(0, min(pred_ymax, label_ymax) - max(pred_ymin, label_ymin))

    pred_area = (pred_xmax - pred_xmin) * (pred_ymax - pred_ymin)
    label_area = (label_xmax - label_xmin) * (label_ymax - label_ymin)
    union_area = pred_area + label_area - inter_area

    iou = inter_area / union_area
    conf= boxes_preds[...,4:5]
    return iou, conf
        

In [ ]:
# Validation Image
df_list=[]
iou_thres= 0.5
for i in range(len(test_df)):
    
    # Taking a test image
    image_tensor, labels, image_id, image_size = test_yds[i]
    boxes_labels = labels[:, 2:]
    boxes_labels[:, [0, 2]] *= target_image_size
    boxes_labels[:, [1, 3]] *= target_image_size
    
    # Predicting 
    with torch.no_grad():
        model_outputs = best_model(image_tensor[None])
        preds = best_model.postprocess(model_outputs, conf_thres=0., multiple_labels_per_box=False)  
        
    # This has the bounding boxes with confidence and class label
    nms_predictions = filter_eval_predictions(preds, confidence_threshold=0.1)
    data= {'image_id':[],'gt_flag':[],'pd_flag':[], 'confidence':[], 'iou':[], 'tp':[], 'fp':[], 'fn':[], 'tn':[]}
    
    
    # Chec if the Ground Truth Bbox is available:
    if boxes_labels.numel()==0:
        # Now check if Predicted Bounding boxes are zero or any got predicted:
        if nms_predictions[0].numel()==0:
#             print(" We dont care about this case")
            data['image_id']= image_id.tolist()
            data['gt_flag'].append(1)
            data['pd_flag'].append(1)
            data['confidence'].append(np.nan)
            data['iou'].append(np.nan)
            data['tp'].append(np.nan)
            data['fn'].append(np.nan)
            data['fp'].append(np.nan)
            data['tn'].append(1)
        elif nms_predictions[0].numel()!=0:
            data['image_id']= image_id.tolist()
            data['gt_flag'].append(1)
            data['pd_flag'].append(0)
            data['confidence'].append(np.nan)
            data['iou'].append(np.nan)
            data['tp'].append(np.nan)
            data['fn'].append(np.nan)
            data['fp'].append(1)
            data['tn'].append(np.nan)
    # this is when the ground truth bounding box is available:
    else:
        # Now we chck if the prediction are done or not:
        # Checking if there are no predictions:
        if nms_predictions[0].numel()==0:
            data['image_id']= image_id.tolist()
            data['gt_flag'].append(0)
            data['pd_flag'].append(1)
            data['confidence'].append(np.nan)
            data['iou'].append(np.nan)
            data['tp'].append(np.nan)
            data['fn'].append(1)
            data['fp'].append(np.nan)
            data['tn'].append(np.nan)
        # Checking ig the prediction is done
        elif nms_predictions[0].numel()!=0:
            # In this case we check for IOU:
            # First we check of the number of predictions are 1 or more:
#             my_iou_list=[]
#             confidence= []
            
            if len(nms_predictions[0])>1:
                for j in range(len(nms_predictions[0])):
                    my_iou, conf= intersection_over_union(nms_predictions[0][j], boxes_labels, box_format= "corners")
#                     print(conf)
                    if my_iou> iou_thres:
                        data['image_id']= image_id.tolist()
                        data['gt_flag'].append(0)
                        data['pd_flag'].append(0)
                        data['confidence'].append(conf[0].tolist())
                        data['iou'].append(my_iou[0][0].tolist())
                        data['tp'].append(1)
                        data['fn'].append(np.nan)
                        data['fp'].append(np.nan)
                        data['tn'].append(np.nan)
                    else:
                        data['image_id']= image_id.tolist()
                        data['gt_flag'].append(0)
                        data['pd_flag'].append(0)
                        data['confidence'].append(conf[0].tolist())
                        data['iou'].append(my_iou[0][0].tolist())
                        data['tp'].append(np.nan)
                        data['fn'].append(np.nan)
                        data['fp'].append(1)
                        data['tn'].append(np.nan)
                        
#                     my_iou_list.append(my_iou)
#                     confidence.append(conf)
                
            else:
                
                my_iou, conf= intersection_over_union(nms_predictions[0], boxes_labels, box_format= "corners")
                
#                 print(conf)
                
                if my_iou> iou_thres:
                    data['image_id']= image_id.tolist()
                    data['gt_flag'].append(0)
                    data['pd_flag'].append(0)
                    data['confidence'].append(conf[0][0].tolist())
                    data['iou'].append(my_iou[0][0].tolist())
                    data['tp'].append(1)
                    data['fn'].append(np.nan)
                    data['fp'].append(np.nan)
                    data['tn'].append(np.nan)
                else:
                    data['image_id']= image_id.tolist()
                    data['gt_flag'].append(0)
                    data['pd_flag'].append(0)
                    data['confidence'].append(conf[0][0].tolist())
                    data['iou'].append(my_iou[0][0].tolist())
                    data['tp'].append(np.nan)
                    data['fn'].append(np.nan)
                    data['fp'].append(1)
                    data['tn'].append(np.nan)
#                 my_iou_list.append(my_iou)
#                 confidence.append(conf)

    df= pd.DataFrame(data)
#     display(df)
    df_list.append(df)
#     break

In [ ]:
final_df= pd.concat(df_list).reset_index().drop(columns='index')
final_df_sort= final_df.sort_values('confidence', ascending=False).reset_index().drop(columns='index')

In [ ]:
final_df_sort['tp_fp_fn'] = np.where((final_df_sort['tp'] ==1.0) , 'TP', 
                        np.where((final_df_sort['fp'] ==1.0) , 'FP', 
                                 np.where((final_df_sort['tn'] ==1.0) , 'TN',
                                          np.where((final_df_sort['fn'] ==1.0) , 'FN', np.nan))))

###############################################

final_df_sort['tp'] = final_df_sort['tp_fp_fn'].apply(lambda x: 1 if x == 'TP' else 0)
final_df_sort['fp'] = final_df_sort['tp_fp_fn'].apply(lambda x: 1 if x == 'FP' else 0)
final_df_sort['fn'] = final_df_sort['tp_fp_fn'].apply(lambda x: 1 if x == 'FN' else 0)
final_df_sort['tn'] = final_df_sort['tp_fp_fn'].apply(lambda x: 1 if x == 'TN' else 0)


In [ ]:
final_df_sort

In [ ]:
from sklearn.metrics import auc

tp = np.cumsum(final_df_sort['tp_fp_fn'] == 'TP')
fp = np.cumsum(final_df_sort['tp_fp_fn'] == 'FP')
fn = np.sum(final_df_sort['tp_fp_fn'] == 'FN')

# Calculate precision and recall at each threshold
precision = tp / (tp + fp)
recall = tp / (tp + fn)

final_df_sort['precision']= precision
final_df_sort['recall']= recall
auc_pr = auc(final_df_sort['recall'], final_df_sort['precision'])

# Or can use the:
auc= torch.trapz(torch.tensor(final_df_sort['precision'].values), torch.tensor(final_df_sort['recall'].values))


In [ ]:
final_df_sort

In [ ]:
auc_pr, auc

In [ ]:
plt.plot(final_df_sort['recall'], final_df_sort['precision'])
plt.title('Precision vs Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')